<a href="https://colab.research.google.com/github/rohit10tiwari/MNIST_Fashion_CNN_Optimized/blob/master/OptimizeCNNModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Create a CNN model and optimise it using Keras tuner**

In [5]:
!pip install keras-tuner

In [6]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
print(tf.__version__)

2.2.0


In [7]:
fashion_mnist=keras.datasets.fashion_mnist

In [8]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [9]:
train_images=train_images/255.0
test_images=test_images/255.0

In [10]:
train_images[0].shape

(28, 28)

In [11]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)


In [12]:
def build_model(hp):
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=64, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values=[3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),                        
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values=[3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units',min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')    
  ])

  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate',values=[1e-2,1e-3])),
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
  
  return model

In [13]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [14]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy', 
                          max_trials=5, directory= 'output', project_name='Mnist Fashion')

In [15]:
tuner_search.search(train_images,train_labels,epochs=3, validation_split=0.1)

Epoch 1/3
1688/1688 [==============================] - 21s 12ms/step - loss: 0.4497 - accuracy: 0.8444 - val_loss: 0.3976 - val_accuracy: 0.8585
Epoch 2/3
1688/1688 [==============================] - 19s 11ms/step - loss: 0.3210 - accuracy: 0.8813 - val_loss: 0.3507 - val_accuracy: 0.8672
Epoch 3/3
1688/1688 [==============================] - 19s 11ms/step - loss: 0.2800 - accuracy: 0.8959 - val_loss: 0.3310 - val_accuracy: 0.8832


Epoch 1/3
1688/1688 [==============================] - 16s 9ms/step - loss: 0.3927 - accuracy: 0.8579 - val_loss: 0.2957 - val_accuracy: 0.8895
Epoch 2/3
1688/1688 [==============================] - 15s 9ms/step - loss: 0.2551 - accuracy: 0.9054 - val_loss: 0.2891 - val_accuracy: 0.8955
Epoch 3/3
1688/1688 [==============================] - 15s 9ms/step - loss: 0.1929 - accuracy: 0.9284 - val_loss: 0.2519 - val_accuracy: 0.9080


Epoch 1/3
1688/1688 [==============================] - 16s 10ms/step - loss: 0.3995 - accuracy: 0.8553 - val_loss: 0.2921 - val_accuracy: 0.8947
Epoch 2/3
1688/1688 [==============================] - 16s 9ms/step - loss: 0.2558 - accuracy: 0.9070 - val_loss: 0.2736 - val_accuracy: 0.8997
Epoch 3/3
1688/1688 [==============================] - 16s 10ms/step - loss: 0.1972 - accuracy: 0.9253 - val_loss: 0.2462 - val_accuracy: 0.9072


Epoch 1/3
1688/1688 [==============================] - 16s 10ms/step - loss: 0.4952 - accuracy: 0.8221 - val_loss: 0.3653 - val_accuracy: 0.8628
Epoch 2/3
1688/1688 [==============================] - 16s 9ms/step - loss: 0.3728 - accuracy: 0.8624 - val_loss: 0.3497 - val_accuracy: 0.8713
Epoch 3/3
1688/1688 [==============================] - 16s 9ms/step - loss: 0.3390 - accuracy: 0.8751 - val_loss: 0.3576 - val_accuracy: 0.8687


Epoch 1/3
1688/1688 [==============================] - 19s 11ms/step - loss: 0.5763 - accuracy: 0.8072 - val_loss: 0.4180 - val_accuracy: 0.8438
Epoch 2/3
1688/1688 [==============================] - 18s 11ms/step - loss: 0.3991 - accuracy: 0.8537 - val_loss: 0.4194 - val_accuracy: 0.8513
Epoch 3/3
1688/1688 [==============================] - 18s 11ms/step - loss: 0.3781 - accuracy: 0.8585 - val_loss: 0.3910 - val_accuracy: 0.8533


INFO:tensorflow:Oracle triggered exit


In [16]:
model=tuner_search.get_best_models(num_models=1)[0]

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 96)        960       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 32)        76832     
_________________________________________________________________
flatten (Flatten)            (None, 15488)             0         
_________________________________________________________________
dense (Dense)                (None, 112)               1734768   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1130      
Total params: 1,813,690
Trainable params: 1,813,690
Non-trainable params: 0
_________________________________________________________________
